In [1]:
# %reset
import pandas as pd
import numpy as np
import string as st
import re
import numpy as np

In [2]:
df = pd.read_excel("Recommended Computers.xlsx")
"CPU dataframe"
cpu_db = pd.read_excel("./Component Databases/CPU Final DB.xlsx",sheet_name = "Processors")

In [3]:
def cpu_cleanup(x):
    "remove spaces, lowercase, and replace roman with arabic numerals"
    x = str.lower(str(x))
    for symbol in [' ','-','™',"®","(r)","(R)","(tm)","(TM)", "+"]:
        x = x.replace(symbol,"")
        
    x = x.replace("iiii","4")
    x = x.replace("iv","4")
    x = x.replace("iii","3")
    x = x.replace("ii","2")

    
    return x

cpu_db['Name'] = cpu_db['Name'].apply(cpu_cleanup)
cpu_db['Primary Name'] = cpu_db['Primary Name'].apply(cpu_cleanup)
cpu_db['Name Modifier'] = cpu_db['Name Modifier'].apply(cpu_cleanup)
cpu_db.drop_duplicates(subset = ["Name"], inplace = True)



In [4]:
"CPU Manipulation part 1"
"""Pulls out the clock speed"""
def processor_manip(tstr):
    include_set_check = set(['0','1','2','3','4','5','6','7','8','9','0']) #for use later
    include_set_convert = set(['0','1','2','3','4','5','6','7','8','9','0','.']) #for use later
    
    tstr = str(tstr)
    for symbol in ['™',"®","(r)","(R)","(tm)","(TM)","+"," ","-"]:
        tstr = tstr.replace(symbol,"")
    
    tstr = tstr.lower()
    return(tstr)

df['machine cpu name'] = df['cpu'].apply(processor_manip)
# df['Minimum', 'Frequency'],df['Minimum', 'mod_cpu'] = list(zip(*((x[0], x[1]) for x in df['Minimum', 'tmp'].values)))
# df = df.drop('tmp', axis = 'columns', level = 1)


In [5]:
"Write function to pick out processor"
"Modify game database CPUs"

def processor_finder(row,cpu_db = None, cpu_family = None, family_min = None):
    "if it has a frequency find the block from which the frequency was pulled"
    freq = row['cpu freq']
    game_cpu = row['machine cpu name']
    family = None          
    for fam in cpu_family:
        if fam in game_cpu:
            family = str(fam)
            break
    "search the cpu db for all the processors in that family and see if it appears"
    
    cpu_list = list(cpu_db[cpu_db['Primary Name'] == family]['Name Modifier'].unique())
    cpu_list.sort(key = len, reverse = True)
    for cpu in cpu_list:
        if cpu in game_cpu:
            return str(family + cpu)

    "check to see if the frequency appears and search by family/frequency"
    if not pd.isna(freq):
        freq_match_row = cpu_db[(cpu_db['Primary Name'] == family) & (abs(cpu_db['CPU Min (GHz)']-freq) <.001)]
        if len(freq_match_row) != 0:
            return freq_match_row.sort_values(by="CPU Min (GHz)")['Name'].iloc[0]

    "return the lowest end processor of the family if only the family name appears"
    if family != None:
        index_loc = family_min.loc[family].index[0]
        return cpu_db.loc[index_loc]['Name']
        
     
    return None 

   

cpu_family = list(cpu_db['Primary Name'].unique())
cpu_family.sort(key = len, reverse = True)
family_min = cpu_db.groupby(by = 'Primary Name')['CPU Min (GHz)'].nsmallest(1)
df['cpu match'] = df.apply(processor_finder, axis=1, cpu_db = cpu_db, 
                                                 cpu_family = cpu_family, family_min = family_min)
    

def freq_fill(row, cpu_db=None):
    cell = row['cpu match']
    if cell[:2] == 'fx':
        cell = cell[:-4]
    if (pd.isna(row['cpu freq'])) & (not pd.isna(row['cpu match'])):
        return cpu_db.loc[cpu_db['Name'] == cell]['CPU Min (GHz)'].iloc[0]
    else:
        return row['cpu freq']
    
def gflops(row, cpu_db=None):
    if not pd.isna(row['cpu match']):
        cell = row['cpu match']
        if cell[:2] == 'fx':
            cell = cell[:-4]
        cores = cpu_db.loc[cpu_db['Name'] == cell]['Cores'].iloc[0]
        freq = row['cpu freq']
        flops = cpu_db.loc[cpu_db['Name'] == cell]['DP Flop Per Cycle'].iloc[0]
        return cores*freq*flops
    

df['cpu freq'] = df.apply(freq_fill, axis = 1, cpu_db = cpu_db)
df["cpu flops"] = df.apply(gflops, axis = 1, cpu_db = cpu_db)
df = df.drop('machine cpu name', axis = 'columns')     


In [6]:
"gpu game data and ss data cleaning"
def gpu_clean(cell, spaces = True):
    if not pd.isna(cell):
        cell = str(cell).lower()
        if spaces == False:
            for symbol in ['-','™',"®","(r)","(R)","(tm)","(TM)","compatible", "compatiable", ",",'–','+']:
                cell = cell.replace(symbol,"") 
        else:
            for symbol in [' ','-','™',"®","(r)","(R)","(tm)","(TM)"]:
                cell = cell.replace(symbol,"")
        return cell
    return cell

def rate_splitter(cell):
    'Used to clean GPU texture rate from gpudb'
    number,unit = cell.split(" ")
    u = unit[0]
    return float(number)* {"G":1,"M":1/1000}[u]

def make_finder(row):
    "cleans up the gpu make from gpudb"
    brand = row['brand mod']
    if row['name mod'][:6] == 'radeon':
        brand = 'radeon'
    if row['name mod'][:7] == 'geforce':
        brand = 'geforce'
    return brand

def name_cleaner(row):
    'finds modifiers to the GPU name like '
    brand = row['brand mod']
    nm = row['name mod'].replace(brand,"")
    return nm

def gpu_simplifier(cell):
    "Appied to the gpudb name this finds shortened names for families of gpus"
    cell_string = str(cell)
    i=1
    while (not cell_string[-i].isdigit()) & (i < len(cell_string)):
        i+=1
    if (i == 1):
        return None
    elif (i==len(cell_string)):
        return None
    else:
        return cell_string[:-i+1]
    
def gpu_mem(cell):
    "input the cell with the onboard gpu memory and it retuns a number in GB"
    num, mod = cell.split()
    if num[0] not in st.digits:
        return None
    return float(num)*{"GB":1,"MB":1/1024}[mod]
    
"build and clean the gpu dataframe"    
gpu_db = pd.read_excel("./Component Databases/new gpu database.xlsx", sheet_name = "Sheet1")
gpu_db = gpu_db[['brand','name','DirectX','OpenCL','OpenGL','Texture Rate','Memory Size']]
gpu_db['brand mod'] = gpu_db['brand'].apply(gpu_clean)
gpu_db['name mod'] = gpu_db["name"].apply(gpu_clean)
gpu_db['tex mod'] = gpu_db['Texture Rate'].apply(rate_splitter)
gpu_db['brand mod'] = gpu_db.apply(make_finder, axis = 1)
gpu_db['name mod'] = gpu_db.apply(name_cleaner,axis = 1)
gpu_db['gpu mem'] = gpu_db['Memory Size'].apply(gpu_mem)

"This block adds copies of gpu names with the garbage at the end of names removed to get at family names"
gpu_db['shortened'] = gpu_db["name mod"].apply(gpu_simplifier)
tmp_db = gpu_db.copy()
tmp_db = tmp_db.dropna(subset = ['shortened'])
tmp_db = tmp_db.sort_values(by = ['tex mod'])
tmp_db['name mod'] = tmp_db['shortened']
tmp_db = tmp_db.drop_duplicates(subset = ['name mod'])
gpu_db = gpu_db.append(tmp_db, sort = True)
gpu_db = gpu_db.drop_duplicates(subset = ['name mod'])
gpu_db = gpu_db.drop(['shortened'], axis = 'columns')



df['gpu machine name'] = df['gpu'].apply(gpu_clean)


In [7]:
def gpu_finder(row,gpu_db = None,gpu_dict = None):
    cell = row['gpu machine name']
    mem_req = row['gpu mem']
    
    if not pd.isna(cell):
        
        for gpu_make in gpu_dict.keys():
            if gpu_make in cell:
                for gpu_model in gpu_dict[gpu_make]: 
                    if gpu_model in cell:
                        return f"{gpu_make} {gpu_model}"    
            
    return None

gpu_dict = {}
for make in gpu_db['brand mod'].unique():
    gpu_dict[make] = sorted(list(gpu_db[gpu_db['brand mod'] == make]['name mod'].unique()),key=len, reverse = True)
    
gpu_dict['nvidia'].extend(gpu_dict['geforce'])
gpu_dict['ati'].extend(gpu_dict['radeon'])
gpu_dict['amd'].extend(gpu_dict['radeon'])


df["gpu matched name"] = df.apply(gpu_finder, axis = 1, gpu_db = gpu_db, gpu_dict = gpu_dict)

print( f" {len(df[df['gpu matched name'].isna()])} missing of {len(df)}.")  

      
"assign metric value based on gpu name"

def metric_return(cell,df = None):
    """returns the correct metric number for the given name"""
    brand, name = cell.split()
    return df['tex mod'].loc[name]

def ram_return(row,df = None):
    "returns the ram if the cell is empty"
    ram_cell = row['gpu mem']
    
    if pd.isnull(ram_cell):
        brand,name = row['gpu matched name'].split()
        return df['gpu mem'].loc[name]
    else:
        return ram_cell

indexed_gpu_db = gpu_db.set_index("name mod")
"find metrics for matched gpu"
df['gpu_tex'] = df['gpu matched name'].apply(metric_return, df = indexed_gpu_db)
df['gpu mem'] = df.apply(ram_return, axis = 1, df = indexed_gpu_db) 
"multiply each metric by the number of gpus"
df['gpu_tex'] = df['gpu_tex']*df['gpu num']
df['gpu mem'] = df['gpu mem']*df['gpu num']
# df.to_excel('gpu.xlsx')

 0 missing of 93.


In [8]:
"save result as excel file"
df.to_excel("preprocessed computer builds.xlsx")

In [9]:
gpu_db

,DirectX,Memory Size,OpenCL,OpenGL,Texture Rate,brand,brand mod,gpu mem,name,name mod,tex mod
0,NaN,512 MB,1.2,4.4,4.800 GTexel/s,AMD,amd,0.5,FirePro 2270,firepro2270,4.80
1,NaN,512 MB,1.2,4.4,4.800 GTexel/s,AMD,amd,0.5,FirePro 2270 PCIe x1,firepro2270pciex1,4.80
2,NaN,System Shared,1.2,4.4,21.72 GTexel/s,AMD,amd,NaN,FirePro A300,fireproa300,21.72
3,NaN,System Shared,1.2,4.4,22.92 GTexel/s,AMD,amd,NaN,FirePro A320,fireproa320,22.92
4,NaN,2 GB,1.2,4.6,68.00 GTexel/s,AMD,amd,2.0,FirePro D300,fireprod300,68.00
5,NaN,3 GB,1.2,4.6,69.60 GTexel/s,AMD,amd,3.0,FirePro D500,fireprod500,69.60
6,NaN,6 GB,1.2,4.6,108.8 GTexel/s,AMD,amd,6.0,FirePro D700,fireprod700,108.80
7,NaN,1024 MB,1.2,4.4,12.00 GTexel/s,AMD,amd,1.0,FirePro M2000,fireprom2000,12.00
8,NaN,1024 MB,1.2,4.4,6.000 GTexel/s,AMD,amd,1.0,FirePro M3900,fireprom3900,6.00
9,NaN,1024 MB,1.2,4.6,21.60 GTexel/s,AMD,amd,1.0,FirePro M4000,fireprom4000,21.60


In [10]:
df

,type,year,month,date,cpu,cpu freq,gpu,gpu mem,gpu num,ram type,...,total cost,internals cost,inernals cost npv,type_dummy,month_dummy,cpu match,cpu flops,gpu machine name,gpu matched name,gpu_tex
0,Entry,2001,July,2001-07-01,celeron,0.800,geforce 2 mx,0.03125,1,pc-133,...,956.00,653.00,634.932806,1,7,celeron220,3.200,geforce2mx,geforce 2mx,0.70
1,Mid,2001,July,2001-07-01,thunderbird,1.333,geforce2 ultra,0.06250,1,pc-133,...,1858.00,1197.00,1163.881423,2,7,thunderbird1333,2.666,geforce2ultra,geforce 2ultra,2.00
2,Dream,2001,July,2001-07-01,pentium 4,1.700,geforce 3,0.06250,1,pc-800,...,3410.00,2356.00,2290.814229,3,7,pentium41.7,3.400,geforce3,geforce 3,1.60
3,Entry,2001,December,2001-12-01,duron,1.000,geforce 2 Pro,0.06250,1,pc-133,...,952.00,654.00,635.905138,1,12,duron1000,2.000,geforce2pro,geforce 2pro,1.60
4,Mid,2001,December,2001-12-01,thunderbird,1.400,geforce 2 ultra,0.06250,1,pc-2100,...,1598.00,1031.00,1002.474308,2,12,thunderbird1400,2.800,geforce2ultra,geforce 2ultra,2.00
5,Dream,2001,December,2001-12-01,pentium 4,2.000,geforce 3,0.06250,1,pc-800,...,3432.00,2340.00,2275.256917,3,12,pentium42.0,4.000,geforce3,geforce 3,1.60
6,Entry,2002,June,2002-06-01,duron,1.000,geforce 4 mx 440,0.06250,1,pc-133,...,835.00,563.00,538.902724,1,6,duron1000,2.000,geforce4mx440,geforce 4mx440,1.10
7,Mid,2002,June,2002-06-01,thunderbird,1.400,geforce 4 ti 4200,0.06250,1,pc-2100,...,1263.00,789.00,755.229572,2,6,thunderbird1400,2.800,geforce4ti4200,geforce 4ti4200,2.00
8,Dream,2002,June,2002-06-01,pentium 4,2.200,geforce 4 ti 4600,0.12500,1,pc-800,...,3201.00,1946.00,1862.708171,3,6,pentium42.2,4.400,geforce4ti4600,geforce 4ti4600,2.40
9,Entry,2002,December,2002-12-01,duron,1.300,geforce 4 mx 440,0.06250,1,pc-133,...,764.00,493.00,471.898833,1,12,duron1300,2.600,geforce4mx440,geforce 4mx440,1.10


In [11]:
df.iloc[73]

type                                 Mid
year                                2013
month                               July
date                 2013-07-01 00:00:00
cpu                        core i5-4670k
cpu freq                             3.4
gpu                       radeon hd 7950
gpu mem                                3
gpu num                                1
ram type                       ddr3-1600
ram mem                                8
hdd storage                         1000
peripherials cost                      0
total cost                          1013
internals cost                      1013
inernals cost npv                748.802
type_dummy                             2
month_dummy                            7
cpu match                    corei54670k
cpu flops                          217.6
gpu machine name            radeonhd7950
gpu matched name           radeon hd7950
gpu_tex                             89.6
Name: 73, dtype: object